In [1]:
# Data handling libraries
import numpy as np
import pandas as pd
import json
import os

import warnings
warnings.filterwarnings('ignore')

In [4]:
# Determine geospacial delimitations for the Primavera Forest 

# Delimitation coordinates
lat1, lon1 = 20.730654, -103.684725
lat2, lon2 = 20.536931, -103.684725
lat3, lon3 = 20.536931, -103.449552
lat4, lon4 = 20.731724, -103.449552
del_lat = [lat1,lat2,lat3,lat4]
del_lon = [lon1,lon2,lon3,lon4]

# Generate a grid for the data points:
lat_numbers = 60
lon_numbers = 60
lat_vector = np.linspace(min(del_lat), max(del_lat), lat_numbers)
lon_vector = np.linspace(min(del_lon), max(del_lon), lon_numbers)
lon,lat = np.meshgrid(lon_vector,lat_vector)

# Have all the coordinates in a vector and in a dataframe
coordinates = [[lat,lon] for lat in lat_vector for lon in lon_vector]
coordinates_df = pd.DataFrame(coordinates, columns=['latitude','longitude'])

In [2]:
# Reduce all the datasets to merge them into one
forest_fire = [pd.read_excel(f'./Serie_historica/Serie_historica_anual_incendios_(20{i}).xlsx') for i in range(10,18)]

In [5]:
f0 = forest_fire[0][['Estado','Municipio', 'Coordenadas', 'Tipo de Incendio', 'Fecha Inicio', 'Causa']]

# Modify the text of each column
def title(x):
    return x.title()

f0.rename(columns={col:col.replace(' ','_').lower() for col in f0.columns}, inplace=True)
f0.municipio = f0.municipio.apply(title)
f0.estado = f0.estado.apply(title)
f0.tipo_de_incendio = f0.tipo_de_incendio.astype('str').apply(title)
f0.causa = f0.causa.astype('str').apply(title)

# We only care for now the fire on Jalisco
f0_jal = f0[f0.estado=='Jalisco']

# Function to separate the coodinates into longitude and latitude columns

def coord_div_lon(x):
    vals = x.split(' - ')[0].split(' ',2)
    return -1*(float(vals[0])+float(vals[1])/60+float(vals[2])/3600)
def coord_div_lat(x):
    vals = x.split(' - ')[1].split(' ')[1:]
    return float(vals[0])+float(vals[1])/60+float(vals[2])/3600

f0_jal['lat'] = f0_jal.coordenadas.apply(coord_div_lat)
f0_jal['lon'] = f0_jal.coordenadas.apply(coord_div_lon)

f0_jal = f0_jal.drop('coordenadas', axis=1)

# Obtain the desired values inside the perimeter:
f0_jal_p = f0_jal[(f0_jal.lat<max(del_lat))&(f0_jal.lat>min(del_lat))&(f0_jal.lon<max(del_lon))&(f0_jal.lon>min(del_lon))]
f0_jal_p.head()

,estado,municipio,tipo_de_incendio,fecha_inicio,causa,lat,lon
2566,Jalisco,El Arenal,Superficial,2010-04-10 00:00:00,Quema De Basureros,20.717778,-103.608056
2569,Jalisco,El Arenal,Superficial,2010-05-30 00:00:00,Fogatas De Paseantes,20.682778,-103.607500
2570,Jalisco,El Arenal,Superficial,2010-06-03 00:00:00,Actividades Agropecuarias,20.706667,-103.633333
2571,Jalisco,El Arenal,Superficial,2010-12-14 13:50:00,Fumadores,20.724167,-103.612500
2679,Jalisco,Tala,Superficial,2010-04-26 00:00:00,Actividades Agropecuarias,20.670278,-103.674722


In [6]:
f1 = forest_fire[1][['Estado','Municipio','Coordenadas', 'Tipo de Incendio', 'Fecha Inicio', 'Causa']]

# Modify the columns for a better manipulation
f1.rename(columns={col:col.replace(' ','_').lower() for col in f1.columns}, inplace=True)

# Modify the text of each column
f1.municipio = f1.municipio.apply(title)
f1.estado = f1.estado.apply(title)
f1.tipo_de_incendio = f1.tipo_de_incendio.astype('str').apply(title)
f1.causa = f1.causa.astype('str').apply(title)

# We only care for the fire on Jalisco
f1_jal = f1[f1.estado=='Jalisco']

# Separate the coodinates into longitude and latitude columns
f1_jal['lat'] = f1_jal.coordenadas.apply(coord_div_lat)
f1_jal['lon'] = f1_jal.coordenadas.apply(coord_div_lon)
f1_jal = f1_jal.drop('coordenadas', axis=1)

# Obtain the desired values inside the perimeter:
f1_jal_p = f1_jal[(f1_jal.lat<max(del_lat))&(f1_jal.lat>min(del_lat))&(f1_jal.lon<max(del_lon))&(f1_jal.lon>min(del_lon))]
f1_jal_p.head()

,estado,municipio,tipo_de_incendio,fecha_inicio,causa,lat,lon
4757,Jalisco,El Arenal,Superficial,2011-01-29 00:00:00,Limpia De Derechos De Via,20.731667,-103.618889
4762,Jalisco,El Arenal,Superficial,2011-03-22 00:00:00,Fumadores,20.713611,-103.622778
4764,Jalisco,El Arenal,Superficial,2011-04-16 00:00:00,Fogatas De Paseantes,20.693139,-103.655611
4765,Jalisco,El Arenal,Superficial,2011-04-20 00:00:00,Fogatas De Paseantes,20.700500,-103.663528
4767,Jalisco,El Arenal,Superficial,2011-04-28 00:00:00,Quema De Basureros,20.705583,-103.630778


In [7]:
f2 = forest_fire[2][['Estado', 'Municipio', 'Coordenadas','Tipo de incendio','Fecha Inicio', 'Causa','Tipo impacto']]
f2.rename(columns={col:col.replace(' ','_').lower() for col in f2.columns}, inplace=True)

def coord_div_lon(x):
    vals = x.split('-')[1].replace(' ','').replace('°','|').replace('´','|').replace('¨','|').split('|')
    return  -1*(float(vals[0])+float(vals[1])/60+float(vals[2])/3600)

def coord_div_lat(x):
    vals = x.split('-')[0].replace(' ','').replace('°','|').replace('´','|').replace('¨','|').split('|')
    return  float(vals[0])+float(vals[1])/60+float(vals[2])/3600

f2_jal = f2[f2.estado=='Jalisco']
f2_jal['lat'] = f2_jal.coordenadas.apply(coord_div_lat)
f2_jal['lon'] = f2_jal.coordenadas.apply(coord_div_lon)
f2_jal = f2_jal.drop('coordenadas', axis=1)
f2_jal_p = f2_jal[(f2_jal.lat<max(del_lat))&(f2_jal.lat>min(del_lat))&(f2_jal.lon<max(del_lon))&(f2_jal.lon>min(del_lon))]
f2_jal_p.head()
#'Detección','Llegada', 'Duración', 'Voluntarios'

,estado,municipio,tipo_de_incendio,fecha_inicio,causa,tipo_impacto,lat,lon
3063,Jalisco,Zapopan,0,2012-01-02 00:00:00,Fumadores,minimo,20.649278,-103.473750
3064,Jalisco,Zapopan,0,2012-01-04 00:00:00,Fumadores,minimo,20.725000,-103.559722
3065,Jalisco,Tala,0,2012-01-08 00:00:00,Fumadores,minimo,20.564444,-103.583056
3072,Jalisco,Zapopan,Superficial,2012-01-20 00:00:00,Fumadores,minimo,20.648889,-103.467222
3074,Jalisco,Zapopan,Superficial,2012-01-21 00:00:00,Quema de basureros,minimo,20.697778,-103.476944


In [8]:
f3 = forest_fire[3][['Estado', 'Municipio', 'Coordenadas', 'Tipo de incendio', 'Fecha Inicio', 'Causa', 'Tipo impacto']]
f3.rename(columns={col:col.replace(' ','_').lower() for col in f3.columns}, inplace=True)

f3_jal = f3[f3.estado=='Jalisco']

f3_jal['lat'] = f3_jal.coordenadas.apply(coord_div_lat)
f3_jal['lon'] = f3_jal.coordenadas.apply(coord_div_lon)

f3_jal = f3_jal.drop('coordenadas', axis=1)
f3_jal_p = f3_jal[(f3_jal.lat<max(del_lat))&(f3_jal.lat>min(del_lat))&(f3_jal.lon<max(del_lon))&(f3_jal.lon>min(del_lon))]
f3_jal_p.head()
# 'Detección','Llegada', 'Duración', 'Voluntarios'

,estado,municipio,tipo_de_incendio,fecha_inicio,causa,tipo_impacto,lat,lon
6341,Jalisco,Zapopan,Superficial,2013-01-10,Fumadores,Minimo,20.717500,-103.521667
6345,Jalisco,Zapopan,Superficial,2013-01-22,Fumadores,Minimo,20.693306,-103.620722
6348,Jalisco,Zapopan,Superficial,2013-01-26,Fumadores,Minimo,20.730444,-103.576167
6355,Jalisco,Zapopan,Superficial,2013-02-05,Actividades Agropecuarias,Minimo,20.696389,-103.603333
6356,Jalisco,Zapopan,Superficial,2013-02-06,Intencional,Minimo,20.708889,-103.575556


In [9]:
f4 = forest_fire[4][['Estado', 'Municipio', 'Coordenadas', 'Tipo de incendio','Fecha Inicio', 'Causa',  'Tipo ']]
f4.rename(columns={col:col.replace(' ','_').lower() for col in f4.columns}, inplace=True)
f4.rename(columns={'tipo_':'tipo_impacto'}, inplace=True)

f4_jal = f4[f4.estado=='Jalisco']

f4_jal['lat'] = f4_jal.coordenadas.apply(coord_div_lat)
f4_jal['lon'] = f4_jal.coordenadas.apply(coord_div_lon)
f4_jal = f4_jal.drop('coordenadas', axis=1)
f4_jal_p = f4_jal[(f4_jal.lat<max(del_lat))&(f4_jal.lat>min(del_lat))&(f4_jal.lon<max(del_lon))&(f4_jal.lon>min(del_lon))]
f4_jal_p.head()
# 'Detección','Llegada', 'Duración', 'Voluntarios'

,estado,municipio,tipo_de_incendio,fecha_inicio,causa,tipo_impacto,lat,lon
2229,Jalisco,Zapopan,Superficial,2014-01-30,Fogatas,Minimo,20.696806,-103.579306
2230,Jalisco,Zapopan,Superficial,2014-01-01,Fumadores,Minimo,20.728400,-103.557611
2231,Jalisco,Zapopan,Superficial,2014-01-23,Fumadores,Minimo,20.728056,-103.556667
2234,Jalisco,Zapopan,Superficial,2014-02-12,Fumadores,Minimo,20.725861,-103.557389
2235,Jalisco,Zapopan,Superficial,2014-02-14,Intencional,Minimo,20.726361,-103.557278


In [10]:
f5 = forest_fire[5][['Estado','Municipio','Coordenadas','Tipo de incendio', 'Fecha Inicio', 'Causa', 'Tipo impacto']]
f5.rename(columns={col:col.replace(' ','_').lower() for col in f5.columns}, inplace=True)

f5_jal = f5[f5.estado=='Jalisco']
f5_jal['lat'] = f5_jal.coordenadas.apply(coord_div_lat)
f5_jal['lon'] = f5_jal.coordenadas.apply(coord_div_lon)
f5_jal = f5_jal.drop('coordenadas', axis=1)
f5_jal_p = f5_jal[(f5_jal.lat<max(del_lat))&(f5_jal.lat>min(del_lat))&(f5_jal.lon<max(del_lon))&(f5_jal.lon>min(del_lon))]
f5_jal_p.head()
# 'Detección','Llegada', 'Duracion', 'Voluntarios'

,estado,municipio,tipo_de_incendio,fecha_inicio,causa,tipo_impacto,lat,lon
1492,Jalisco,Zapopan,Superficial,2015-01-03 00:00:00,Actividades Agropecuarias,Minimo,20.703444,-103.553664
1496,Jalisco,Zapopan,Superficial,2015-01-09 00:00:00,Fogatas,Minimo,20.727278,-103.602833
1498,Jalisco,Tala,Superficial,2015-01-12 00:00:00,Actividades Agropecuarias,Minimo,20.685600,-103.636000
1519,Jalisco,Tala,Superficial,2015-02-03 00:00:00,Actividades Agropecuarias,Minimo,20.664736,-103.681167
1531,Jalisco,Zapopan,Superficial,2015-03-02 00:00:00,Quema de basureros,Minimo,20.648556,-103.469083


In [11]:
# HACE FALTA COORDENADAS

def change_to_string(x):
    x=str(x).replace('°','').replace('-','').replace("'",'').replace('"','')
    return float(x)

elements = ['Grados', 'Minutos','Segundos']
for element in elements:
    forest_fire[6][element]=forest_fire[6][element].apply(change_to_string)
    forest_fire[6][f'{element}.1']=forest_fire[6][f'{element}.1'].apply(change_to_string)
    
forest_fire[6]['lat']=forest_fire[6]['Grados'] + forest_fire[6]['Minutos']/60+ forest_fire[6]['Segundos']/3600
forest_fire[6]['lon']=-1*(forest_fire[6]['Grados.1'] + forest_fire[6]['Minutos.1']/60+ forest_fire[6]['Segundos.1']/3600)

f6 = forest_fire[6][['Estado', 'Municipio', 'Tipo de incendio', 'Fecha Inicio', 'Causa', 'Tipo impacto', 'lat','lon']]
# 'Detección', 'Llegada', 'Duración', 'Voluntarios'
f6.rename(columns={col:col.replace(' ','_').lower() for col in f6.columns}, inplace=True)
f6_jal = f6[f6.estado=='Jalisco']
f6_jal_p = f6_jal[(f6_jal.lat<max(del_lat))&(f6_jal.lat>min(del_lat))&(f6_jal.lon<max(del_lon))&(f6_jal.lon>min(del_lon))]
f6_jal_p.head()

,estado,municipio,tipo_de_incendio,fecha_inicio,causa,tipo_impacto,lat,lon
2853,Jalisco,Tala,Superficial,2016-01-10,Actividades Agropecuarias,Impacto Mínimo,20.591889,-103.670167
2865,Jalisco,Zapopan,Superficial,2016-01-21,Fumadores,Impacto Mínimo,20.619167,-103.506389
2869,Jalisco,Zapopan,Superficial,2016-01-28,Fumadores,Impacto Mínimo,20.652500,-103.464611
2879,Jalisco,Zapopan,Superficial,2016-02-02,Fumadores,Impacto Mínimo,20.649628,-103.463900
2880,Jalisco,Zapopan,Superficial,2016-02-06,Fumadores,Impacto Mínimo,20.685300,-103.578478


In [12]:
# HACE FALTA COORDENADAS
for element in elements:
    forest_fire[7][element]=forest_fire[7][element].apply(change_to_string)
    forest_fire[7][f'{element}.1']=forest_fire[7][f'{element}.1'].apply(change_to_string)
    
forest_fire[7]['lat']=forest_fire[7]['Grados'] + forest_fire[7]['Minutos']/60+ forest_fire[7]['Segundos']/3600
forest_fire[7]['lon']=-1*(forest_fire[7]['Grados.1'] + forest_fire[7]['Minutos.1']/60+ forest_fire[7]['Segundos.1']/3600)

f7 = forest_fire[7][['Estado', 'Municipio', 'Tipo de incendio', 'Fecha Inicio', 'Causa', 'Tipo impacto', 'lat','lon']]
f7.rename(columns={col:col.replace(' ','_').lower() for col in f7.columns}, inplace=True)
f7_jal = f7[f7.estado=='Jalisco']
f7_jal_p = f7_jal[(f7_jal.lat<max(del_lat))&(f7_jal.lat>min(del_lat))&(f7_jal.lon<max(del_lon))&(f7_jal.lon>min(del_lon))]
f7_jal_p.head()
# 'Detección', 'Llegada', 'Duración', 'Voluntarios'

,estado,municipio,tipo_de_incendio,fecha_inicio,causa,tipo_impacto,lat,lon
3524,Jalisco,Tala,Superficial,2017-01-17,Limpias de derecho de vía,Impacto Mínimo,20.678139,-103.668650
3530,Jalisco,El Arenal,Superficial,2017-01-20,Limpias de derecho de vía,Impacto Mínimo,20.701958,-103.619469
3539,Jalisco,Zapopan,Superficial,2017-02-02,Intencional,Impacto Mínimo,20.645917,-103.485528
3540,Jalisco,Tala,Superficial,2017-02-03,Actividades Agropecuarias,Impacto Mínimo,20.673139,-103.673500
3548,Jalisco,Zapopan,Superficial,2017-02-11,Fumadores,Impacto Mínimo,20.653322,-103.460319


In [14]:
fire_jal_p = pd.concat([f0_jal_p, f1_jal_p, f2_jal_p, f3_jal_p, f4_jal_p, f5_jal_p, f6_jal_p, f7_jal_p],axis=0)
fire_jal_p = fire_jal_p.reset_index(drop=True)[['lat','lon','fecha_inicio','tipo_de_incendio','tipo_impacto','causa']]

def impacto(x):
    x = str(x)
    return x.replace('Impacto','').lower().replace(' ','').replace('í','i').title()
fire_jal_p.tipo_impacto = fire_jal_p.tipo_impacto.apply(impacto)

def fecha(x):
    return str(x).split(' ')[0]
fire_jal_p.fecha_inicio = pd.to_datetime(fire_jal_p.fecha_inicio.apply(fecha))

def year(x):
    return int(str(x).split('-',1)[0])

fire_jal_p['year'] = fire_jal_p.fecha_inicio.apply(year)
fire_jal_p.causa = fire_jal_p.causa.str.replace('Limpias', 'limpia').apply(title)
fire_jal_p.causa = fire_jal_p.causa.str.replace('Derechos', 'Derecho')
fire_jal_p.causa = fire_jal_p.causa.str.replace('Via', 'Vía')
fire_jal_p.causa = fire_jal_p.causa.str.replace('Otras Causas', 'No Determinadas')
fire_jal_p.causa = fire_jal_p.causa.str.replace(' De Paseantes', '')

# Save to csv
fire_jal_p.to_csv('./Serie_historica/fire_jal_p.csv')